In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
data = pd.read_csv("preprocessed_data.csv")

In [60]:
data.head()

,drug_name,age,gender,condition,dosage,other_conditions,other_drugs_taken,ratings,rating_sentiment,benefits,side_effects,comments
0,abilify,26,female,bipolar disorder,15mg taken daily for the period of 12 weeks,NaN,"cymbalta, 90mg/daily",3,negative,i did not notice any benefit at all supposedly...,a very uncomfortable inner restlessness wa the...,i wa prescribed abilify 15mg daily to assist i...
1,abilify,29,female,bipolar,10mg taken 1/day for the period of 8 mos,"bipolar mood disorder, acne, asthma","zoloft, wellbutrin, clozepam, tretinoin, clind...",8,positive,i had severe depression with agitation and mix...,i became drowsy however with adequate sleep 10...,the abilify decreased the need for daily klono...
2,abilify,43,female,depressionxiety,2 mg taken daily for the period of 3 months,anxiety,lexapro,10,positive,within 1 week of taking the cocktail of abilif...,no side effect have been noticed,i take one pill of each 1st thing in the am i ...
3,abilify,50,female,depression not resolved with antidepressant drugs,started out at 5mg the 10 & last 15mg taken d...,"add, depression, poss. bipolar type 2, ptsd, a...","vyvance, ativan lisinopril, zantac, viville do...",2,negative,while on abilify i can honestly say the depres...,but it caused memory loss and again an inciden...,i am only taking ativan getting psychological ...
4,abilify,50,male,bipolar,2mg to start taken once daily for the period ...,anxiety,klonopin,2,negative,due to the short time taking drug,headache first morning at 4am that wa relieved...,not much to tell i wa just starting treatment ...


In [61]:
data = data.fillna('')

In [62]:
data['numeric_rating'] = data['rating_sentiment'].apply(lambda x:1 if x=='positive' else 0 if x=='neutral' else 2)

In [63]:
data.head()
x = data['comments']
y = data['numeric_rating']

In [64]:
data['comments'].isnull().values.any()

False

In [65]:
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 42)
print(f'Rows in train set: {len(x_train)}\nRows in test set: {len(x_test)}')

Rows in train set: 3121
Rows in test set: 551


In [66]:
#Splitting the dataset into the Training set and Valid set
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.15, random_state = 42)
print(f'Rows in train set: {len(x_train)}\nRows in valid set: {len(x_valid)}')

Rows in train set: 2652
Rows in valid set: 469


In [67]:
from sklearn.impute import SimpleImputer


class ModifiedSimpleImputer(SimpleImputer):
    def transform(self, X):
        return super().transform(X).flatten()
    
imp = ModifiedSimpleImputer(strategy='constant')

In [68]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
cv = CountVectorizer(max_features=2500)

x_train = cv.fit_transform(x_train).toarray()
x_valid = cv.fit_transform(x_valid).toarray()
x_test = cv.fit_transform(x_test).toarray()

In [81]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [82]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(learning_rate=0.01, min_child_weight=3, n_estimators=300, reg_alpha = 0.3, 
                                 subsample=0.8, random_state=42, gamma=5, max_depth=2)
xgb_classifier.fit(x_train,y_train,early_stopping_rounds=40, eval_set=[(x_valid, y_valid)])

C:\Users\ashis\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.09473
[1]	validation_0-mlogloss:1.09101
[2]	validation_0-mlogloss:1.08742
[3]	validation_0-mlogloss:1.08373
[4]	validation_0-mlogloss:1.08015
[5]	validation_0-mlogloss:1.07708
[6]	validation_0-mlogloss:1.07371
[7]	validation_0-mlogloss:1.07028
[8]	validation_0-mlogloss:1.06686
[9]	validation_0-mlogloss:1.06371
[10]	validation_0-mlogloss:1.06054
[11]	validation_0-mlogloss:1.05731
[12]	validation_0-mlogloss:1.05456
[13]	validation_0-mlogloss:1.05150
[14]	validation_0-mlogloss:1.04855
[15]	validation_0-mlogloss:1.04561
[16]	validation_0-mlogloss:1.04308
[17]	validation_0-mlogloss:1.04025
[18]	validation_0-mlogloss:1.03747
[19]	validation_0-mlogloss:1.03473
[20]	validation_0-mlogloss:1.03208
[21]	validation_0-mlogloss:1.02950
[22]	validation_0-mlogloss:1.02697
[23]	validation_0-mlogloss:1.02443
[24]	validation_0-mlogloss:1.02185
[25]	validation_0-mlogloss:1.01948
[26]	validation_0-mlogloss:1.01715
[27]	validation_0-mlogloss:1.01469
[28]	validation_0-mlogloss:1.0

[231]	validation_0-mlogloss:0.89973
[232]	validation_0-mlogloss:0.89968
[233]	validation_0-mlogloss:0.89970
[234]	validation_0-mlogloss:0.89968
[235]	validation_0-mlogloss:0.89967
[236]	validation_0-mlogloss:0.89969
[237]	validation_0-mlogloss:0.89967
[238]	validation_0-mlogloss:0.89968
[239]	validation_0-mlogloss:0.89969
[240]	validation_0-mlogloss:0.89970
[241]	validation_0-mlogloss:0.89970
[242]	validation_0-mlogloss:0.89973
[243]	validation_0-mlogloss:0.89971
[244]	validation_0-mlogloss:0.89974
[245]	validation_0-mlogloss:0.89971
[246]	validation_0-mlogloss:0.89980
[247]	validation_0-mlogloss:0.89980
[248]	validation_0-mlogloss:0.89979
[249]	validation_0-mlogloss:0.89977
[250]	validation_0-mlogloss:0.89980
[251]	validation_0-mlogloss:0.89981
[252]	validation_0-mlogloss:0.89981
[253]	validation_0-mlogloss:0.89979
[254]	validation_0-mlogloss:0.89980
[255]	validation_0-mlogloss:0.89983
[256]	validation_0-mlogloss:0.89984
[257]	validation_0-mlogloss:0.89988
[258]	validation_0-mlogloss:

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=5, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=3,
              missing=nan, monotone_constraints='()', n_estimators=300,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=42, reg_alpha=0.3, ...)

In [83]:
#Visualizing the performace of the model
y_pred_train = xgb_classifier.predict(x_train)

#Printing the classification_report
print(classification_report(y_train, y_pred_train))

#Printing the confusion_matrix
cm = confusion_matrix(y_train , y_pred_train)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_train , y_train)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       393
           1       0.68      1.00      0.81      1785
           2       0.83      0.03      0.06       474

    accuracy                           0.68      2652
   macro avg       0.50      0.34      0.29      2652
weighted avg       0.61      0.68      0.55      2652

[[   0  390    3]
 [   0 1785    0]
 [   0  459   15]]
Accuracy of training set 67.87330316742081%


C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
#Visualizing the performace of the model
y_pred_test = xgb_classifier.predict(x_test)

#Printing the classification_report
print(classification_report(y_test, y_pred_test))

#Printing the confusion_matrix
cm = confusion_matrix(y_test , y_pred_test)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_test , y_test)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.69      1.00      0.82       381
           2       0.00      0.00      0.00       107

    accuracy                           0.69       551
   macro avg       0.23      0.33      0.27       551
weighted avg       0.48      0.69      0.57       551

[[  0  63   0]
 [  0 381   0]
 [  0 107   0]]
Accuracy of training set 69.1470054446461%


C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
from sklearn.linear_model import LogisticRegression
log_classifier = LogisticRegression(random_state=0).fit(x_train,y_train)

C:\Users\ashis\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [86]:
#Visualizing the performace of the model
y_pred_train = log_classifier.predict(x_train)

#Printing the classification_report
print(classification_report(y_train, y_pred_train))

#Printing the confusion_matrix
cm = confusion_matrix(y_train , y_pred_train)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_train , y_train)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       0.99      0.79      0.88       393
           1       0.92      1.00      0.96      1785
           2       0.99      0.84      0.91       474

    accuracy                           0.94      2652
   macro avg       0.97      0.88      0.92      2652
weighted avg       0.94      0.94      0.94      2652

[[ 311   81    1]
 [   2 1780    3]
 [   1   74  399]]
Accuracy of training set 93.89140271493213%


In [88]:
from sklearn import svm
svm_classifier = svm.SVC(decision_function_shape='ovo')
svm_classifier.fit(x_train,y_train)

SVC(decision_function_shape='ovo')

In [89]:
#Visualizing the performace of the model
y_pred_train = svm_classifier.predict(x_train)

#Printing the classification_report
print(classification_report(y_train, y_pred_train))

#Printing the confusion_matrix
cm = confusion_matrix(y_train , y_pred_train)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_train , y_train)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       1.00      0.15      0.26       393
           1       0.72      1.00      0.84      1785
           2       1.00      0.26      0.41       474

    accuracy                           0.74      2652
   macro avg       0.91      0.47      0.50      2652
weighted avg       0.81      0.74      0.68      2652

[[  59  334    0]
 [   0 1785    0]
 [   0  350  124]]
Accuracy of training set 74.2081447963801%


In [90]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=0).fit(x_train,y_train)

In [91]:
#Visualizing the performace of the model
y_pred_train = rf_classifier.predict(x_train)

#Printing the classification_report
print(classification_report(y_train, y_pred_train))

#Printing the confusion_matrix
cm = confusion_matrix(y_train , y_pred_train)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_train , y_train)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       393
           1       0.99      1.00      0.99      1785
           2       1.00      0.97      0.98       474

    accuracy                           0.99      2652
   macro avg       0.99      0.98      0.99      2652
weighted avg       0.99      0.99      0.99      2652

[[ 387    6    0]
 [   3 1782    0]
 [   2   14  458]]
Accuracy of training set 99.05731523378583%


In [113]:
# TF IDF
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(data['comments'], data['numeric_rating'], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size = 0.15, random_state = 42)
print(f'Rows in train set: {len(x_train)}\nRows in valid set: {len(x_valid)}')

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()

tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)
tf_x_valid = vectorizer.fit_transform(X_valid)

Train:  (2754,) (2754,) Test:  ((918,), (918,))
Rows in train set: 2652
Rows in valid set: 469


In [119]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(learning_rate=0.01, min_child_weight=3, n_estimators=300, reg_alpha = 0.3, 
                                 subsample=0.8, random_state=42, gamma=5, max_depth=2)
xgb_classifier.fit(tf_x_train,Y_train,early_stopping_rounds=40, eval_set=[(x_valid, y_valid)])

#Visualizing the performace of the model
y_pred_train = xgb_classifier.predict(tf_x_train)
#Printing the classification_report
print(classification_report(Y_train, y_pred_train))

#Printing the confusion_matrix
cm = confusion_matrix(Y_train , y_pred_train)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_train , Y_train)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

[0]	validation_0-mlogloss:1.09466
[1]	validation_0-mlogloss:1.09073
[2]	validation_0-mlogloss:1.08843
[3]	validation_0-mlogloss:1.08629
[4]	validation_0-mlogloss:1.08268
[5]	validation_0-mlogloss:1.07900
[6]	validation_0-mlogloss:1.07533
[7]	validation_0-mlogloss:1.07188
[8]	validation_0-mlogloss:1.06853
[9]	validation_0-mlogloss:1.06555
[10]	validation_0-mlogloss:1.06216
[11]	validation_0-mlogloss:1.06009
[12]	validation_0-mlogloss:1.05714
[13]	validation_0-mlogloss:1.05508
[14]	validation_0-mlogloss:1.05332
[15]	validation_0-mlogloss:1.05035
[16]	validation_0-mlogloss:1.04726
[17]	validation_0-mlogloss:1.04432
[18]	validation_0-mlogloss:1.04260
[19]	validation_0-mlogloss:1.03996
[20]	validation_0-mlogloss:1.03717
[21]	validation_0-mlogloss:1.03434
[22]	validation_0-mlogloss:1.03156
[23]	validation_0-mlogloss:1.02917
[24]	validation_0-mlogloss:1.02654
[25]	validation_0-mlogloss:1.02433
[26]	validation_0-mlogloss:1.02444
[27]	validation_0-mlogloss:1.02324
[28]	validation_0-mlogloss:1.0

[231]	validation_0-mlogloss:0.90046
[232]	validation_0-mlogloss:0.90061
[233]	validation_0-mlogloss:0.90107
[234]	validation_0-mlogloss:0.90097
[235]	validation_0-mlogloss:0.90082
[236]	validation_0-mlogloss:0.90073
[237]	validation_0-mlogloss:0.90064
[238]	validation_0-mlogloss:0.90053
[239]	validation_0-mlogloss:0.90044
[240]	validation_0-mlogloss:0.90068
[241]	validation_0-mlogloss:0.90058
[242]	validation_0-mlogloss:0.90047
[243]	validation_0-mlogloss:0.90035
[244]	validation_0-mlogloss:0.90026
[245]	validation_0-mlogloss:0.90019
[246]	validation_0-mlogloss:0.90007
[247]	validation_0-mlogloss:0.90002
[248]	validation_0-mlogloss:0.89999
[249]	validation_0-mlogloss:0.89996
[250]	validation_0-mlogloss:0.89986
[251]	validation_0-mlogloss:0.89975
[252]	validation_0-mlogloss:0.89968
[253]	validation_0-mlogloss:0.89972
[254]	validation_0-mlogloss:0.89964
[255]	validation_0-mlogloss:0.89958
[256]	validation_0-mlogloss:0.89952
[257]	validation_0-mlogloss:0.89944
[258]	validation_0-mlogloss:

C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ashis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [103]:
from sklearn.linear_model import LogisticRegression
log_classifier = LogisticRegression(random_state=0).fit(tf_x_train,Y_train)

#Visualizing the performace of the model
y_pred_train = log_classifier.predict(tf_x_train)

#Printing the classification_report
print(classification_report(Y_train, y_pred_train))

#Printing the confusion_matrix
cm = confusion_matrix(Y_train , y_pred_train)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_train , Y_train)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       0.97      0.08      0.15       393
           1       0.74      1.00      0.85      1848
           2       0.93      0.44      0.59       513

    accuracy                           0.76      2754
   macro avg       0.88      0.50      0.53      2754
weighted avg       0.81      0.76      0.70      2754

[[  31  350   12]
 [   1 1842    5]
 [   0  289  224]]
Accuracy of training set 76.1437908496732%


In [106]:
from sklearn import svm
svm_classifier = svm.SVC(decision_function_shape='ovo')
svm_classifier.fit(tf_x_train,Y_train)

#Visualizing the performace of the model
y_pred_train = svm_classifier.predict(tf_x_train)

#Printing the classification_report
print(classification_report(Y_train, y_pred_train))

#Printing the confusion_matrix
cm = confusion_matrix(Y_train , y_pred_train)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_train , Y_train)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       1.00      0.37      0.54       393
           1       0.84      1.00      0.91      1848
           2       1.00      0.78      0.88       513

    accuracy                           0.87      2754
   macro avg       0.95      0.72      0.78      2754
weighted avg       0.89      0.87      0.85      2754

[[ 145  247    1]
 [   0 1848    0]
 [   0  111  402]]
Accuracy of training set 86.96441539578794%


In [107]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=0).fit(tf_x_train,Y_train)
#Visualizing the performace of the model
y_pred_train = rf_classifier.predict(tf_x_train)

#Printing the classification_report
print(classification_report(Y_train, y_pred_train))

#Printing the confusion_matrix
cm = confusion_matrix(Y_train , y_pred_train)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_train , Y_train)
acc_score = acc_score*100
print('Accuracy of training set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       393
           1       0.99      1.00      0.99      1848
           2       1.00      0.97      0.99       513

    accuracy                           0.99      2754
   macro avg       1.00      0.98      0.99      2754
weighted avg       0.99      0.99      0.99      2754

[[ 385    8    0]
 [   0 1848    0]
 [   0   14  499]]
Accuracy of training set 99.20116194625999%
